In [6]:
import pandas as pd
import numpy as np
import pathlib
import json
import re

tables_num = ['I.','II.','III.','IV.','V.', 'VI.']
subsets = ['1.','1.1','2.','2.1','3.','3.1','4.','5.','6.']

# Functions
def all_null(row):
    if len(row.unique())==1:
        if np.isnan(row.unique()[0]):
            return 'drop'
    return 'keep'

def sections(x):
    if (type(x)!=float)&(type(x)!=int):
        for i in tables_num:
            #print(x, i)
            if x.split(' ')[0]==i:
                return i
    return np.nan

def check_empty(d):
    l_t = []
    if d.shape[0]==0:
        for i in range(d.shape[1]):
            l_t.append(np.nan)
        d.loc[0] = l_t
    return d

def sub_sections(x):
    if (type(x)!=float)&(type(x)!=int):
        for i in subsets:
            #print x, i
            if x.split(' ')[0]==i:
                return i
    return np.nan

def all_6_nan(row):
    row = row[1:]
    if len(row.unique())==1:
        return 'drop'
    else:
        return 'keep'
    
def put_nulls(df):
    for col in df:
        df[col]=df[col].str.replace(r"^(\[)(.*?)(\])$",'null')
    return df

In [7]:
def crea_json(ruta):
    #df = pd.read_excel('Data/Merino declaracion_intereses.xlsx',index_col=0, header=None)
    df = pd.read_excel(ruta,index_col=0, header=None)
    df.reset_index(inplace=True, drop=True)

    df['act'] = df.apply(lambda row: all_null(row), axis=1)
    df = df[df['act']=='keep']
    df.reset_index(inplace=True, drop=True)

    # SECCION I *****************************************************************
    df['section'] = df[1].apply(lambda x: sections(x))   
    x='drop'
    x_l=[]
    for i in df['section'].tolist():
        if type(i)==str:
            x=i
        x_l.append(x)
    df['section']=x_l
    sec_1 = df[df['section']=='I.']
    sec_1.reset_index(inplace=True, drop=True)
    sec_1_1 = sec_1.iloc[::,:2:].T
    sec_1_1.drop([0,1], axis=1, inplace=True)
    sec_1_1.columns = sec_1_1.loc[1]
    sec_1_1.drop(index=1, inplace=True)
    sec_1_1.reset_index(inplace=True, drop=True)
    sec_1_2 =sec_1.iloc[2::,5:7:].T
    sec_1.iloc[2::,5:7:]
    sec_1_2.columns = sec_1_2.loc[6]
    sec_1_2.drop(index=6, inplace=True)
    sec_1_2.reset_index(inplace=True, drop=True)
    sec_1_final = pd.concat([sec_1_1,sec_1_2], axis=1)
    sec_2 = df[df['section']=='II.'].copy(deep=True)
    sec_2.reset_index(inplace=True, drop=True)

    # SECCION II ************************************************************************
    sec_2['section'] = sec_2[1].apply(lambda x: sub_sections(x))
    x='drop'
    x_l=[]
    for i in sec_2['section'].tolist():
        if type(i)==str:
            x=i
        x_l.append(x)
    sec_2['section']=x_l
    sec_2 = sec_2[sec_2['section']!='drop']
    sec_2_1 = sec_2[sec_2['section']=='1.']
    sec_2_1.reset_index(inplace=True, drop=True)
    sec_2_1.columns = sec_2_1.loc[2]
    sec_2_1 = sec_2_1.iloc[4::,:-3:].copy(deep=True)
    col2_1=[u'Nombre de la empresa',
     u'Sector econ\xf3mico de la empresa',
     u'Pa\xeds en el que est\xe1 constituida la empresa',
     u'Tipo de participaci\xf3n, cargo o funci\xf3n que desempe\xf1\xf3 o desempe\xf1a el declarante en la empresa',
     u'La participaci\xf3n es:',
     u'Si la participaci\xf3n es remunerada, indique el monto anual total de la remuneraci\xf3n \n(incluyendo impuestos)']
    sec_2_1 = sec_2_1[col2_1]
    sec_2_1 = check_empty(sec_2_1)
    sec_2_1.reset_index(inplace=True, drop=True)
    put_nulls(sec_2_1)
    
    sec_2_1_1 = sec_2[sec_2['section']=='1.1']
    sec_2_1_1.reset_index(inplace=True, drop=True)
    sec_2_1_1.columns = sec_2_1_1.loc[2]
    sec_2_1_1 = sec_2_1_1.iloc[4::,:-3:].copy(deep=True)
    col2_1_1=[u'Parentesco (relaci\xf3n entre responsable de participaci\xf3n y el declarante)',
     u'Sector econ\xf3mico de la empresa',
     u'Descripci\xf3n de la actividad econ\xf3mica de la empresa',
     u'Pa\xeds en el que est\xe1 constituida la empresa',
     u'Tipo de participaci\xf3n, cargo o funci\xf3n que desempe\xf1\xf3 o desempe\xf1a el declarante en la empresa',
     u'Fecha desde la que desempe\xf1a el cargo o funci\xf3n',
     u'\xbfLa participaci\xf3n es voluntaria o remunerada?']
    sec_2_1_1 = sec_2_1_1[col2_1_1]
    sec_2_1_1 = check_empty(sec_2_1_1)
    sec_2_1_1.reset_index(inplace=True, drop=True)
    put_nulls(sec_2_1_1)
    
    sec_2_2 = sec_2[sec_2['section']=='2.']
    sec_2_2.reset_index(inplace=True, drop=True)
    sec_2_2.columns = sec_2_2.loc[2]
    sec_2_2 = sec_2_2.iloc[4:-1:,:-3:].copy(deep=True)
    col2_2=[u'Nombre de la empresa o sociedad en la que posee participaci\xf3n accionaria              ',
     u'Sector econ\xf3mico de la empresa',
     u'Pa\xeds en el que est\xe1 constituida la empresa',
     u'Antig\xfcedad de la participaci\xf3n accionaria declarada (a\xf1os)',
     u'Porcentaje de la empresa que representan las acciones declaradas o n\xfamero de acciones pose\xeddas',
     u'Las acciones declaradas representan una participaci\xf3n mayoritaria o de control. \n(Ver nota 1)']
    sec_2_2 = sec_2_2[col2_2]
    sec_2_2 = check_empty(sec_2_2)
    sec_2_2.reset_index(inplace=True, drop=True)
    put_nulls(sec_2_2)
    
    sec_2_2_1 = sec_2[sec_2['section']=='2.1']
    sec_2_2_1.reset_index(inplace=True, drop=True)
    sec_2_2_1.columns = sec_2_2_1.loc[2]
    sec_2_2_1 = sec_2_2_1.iloc[4:-1:,:-4:].copy(deep=True)
    col2_2_1=[u'Parentesco (relaci\xf3n entre responsable de participaci\xf3n y el declarante)',
     u'Sector econ\xf3mico de la empresa',
     u'Descripci\xf3n de la actividad econ\xf3mica de la empresa',
     u'Pa\xeds en el que est\xe1 constituida la empresa',
     u'Las acciones declaradas representan una participaci\xf3n mayoritaria o de control. \n(Ver nota 2)']
    sec_2_2_1 = sec_2_2_1[col2_2_1]
    sec_2_2_1 = check_empty(sec_2_2_1)
    sec_2_2_1.reset_index(inplace=True, drop=True)
    put_nulls(sec_2_2_1)
    
    sec_2_3 = sec_2[sec_2['section']=='3.']
    sec_2_3.reset_index(inplace=True, drop=True)
    sec_2_3.columns = sec_2_3.loc[2]
    sec_2_3 = sec_2_3.iloc[4:-1:,:-3:].copy(deep=True)
    col2_3=[u'Nombre del acreedor o entidad con la que se tiene la obligaci\xf3n financiera',
     u'Tipo de obligaci\xf3n financiera',
     u'Fecha en la que se contrajo la obligaci\xf3n financiera (DD/MM/AA)       ',
     u'Moneda en que fue adquirida la obligaci\xf3n financiera',
     u'Monto o valor original de la obligaci\xf3n financiera',
     u'Monto o valor actual de la obligaci\xf3n financiera al momento de presentar esta declaraci\xf3n']
    sec_2_3 = sec_2_3[col2_3]
    sec_2_3 = check_empty(sec_2_3)
    sec_2_3.reset_index(inplace=True, drop=True)
    put_nulls(sec_2_3)
    
    sec_2_3_1 = sec_2[sec_2['section']=='3.1']
    sec_2_3_1.reset_index(inplace=True, drop=True)
    sec_2_3_1.columns = sec_2_3_1.loc[2]
    sec_2_3_1 = sec_2_3_1.iloc[4:-1:,:-4:].copy(deep=True)
    col2_3_1=[u'Parentesco (relaci\xf3n entre poseedor de obligaci\xf3n financiera y el declarante)',
     u'Nombre del acreedor o entidad con la que se tiene la obligaci\xf3n financiera',
     u'Tipo de obligaci\xf3n financiera',
     u'Fecha en la que se contrajo la obligaci\xf3n financiera (DD/MM/AA)       ']
    sec_2_3_1 = sec_2_3_1[col2_3_1]
    sec_2_3_1 = check_empty(sec_2_3_1)
    sec_2_3_1.reset_index(inplace=True, drop=True)
    put_nulls(sec_2_3_1)
    
    sec_2_4 = sec_2[sec_2['section']=='4.']
    sec_2_4.reset_index(inplace=True, drop=True)
    sec_2_4.columns = sec_2_4.loc[2]
    sec_2_4 = sec_2_4.iloc[4:-1:,:-3:].copy(deep=True)
    col2_4=[u'Tipo de bien inmueble',
     u'Titular(es) del bien inmueble       ',
     u'Pa\xeds en el que se encuentra el inmueble        ',
     u'A\xf1o de adquisici\xf3n del inmueble',
     u'Municipio / delegaci\xf3n en el que se encuentra el inmueble',
     u'Superficie en metros cuadrados del bien declarado']
    sec_2_4 = sec_2_4[col2_4]
    sec_2_4 = check_empty(sec_2_4)
    sec_2_4.reset_index(inplace=True, drop=True)
    put_nulls(sec_2_4)
    
    sec_2_5 = sec_2[sec_2['section']=='5.']
    sec_2_5.reset_index(inplace=True, drop=True)
    sec_2_5.columns = sec_2_5.loc[2]
    sec_2_5 = sec_2_5.iloc[4:-1:,:-6:].copy(deep=True)
    col2_5=[u'Nombre o raz\xf3n social de la entidad relacionada con el inter\xe9s declarado','nan',
    u'Titular(es) del inter\xe9s declarado       ','nan',
    u'Descripci\xf3n de las caracter\xedsticas y condiciones del inter\xe9s econ\xf3mico o financiero                                ']
    if(len(sec_2_5.columns)==len(col2_5)):
        sec_2_5.columns = col2_5
    col2_5=[u'Nombre o raz\xf3n social de la entidad relacionada con el inter\xe9s declarado',
     u'Titular(es) del inter\xe9s declarado       ',
     u'Descripci\xf3n de las caracter\xedsticas y condiciones del inter\xe9s econ\xf3mico o financiero                                ']
    sec_2_5 = sec_2_5[col2_5]
    sec_2_5 = check_empty(sec_2_5)
    sec_2_5.reset_index(inplace=True, drop=True)
    put_nulls(sec_2_5)

    # SECCION III ****************************************************************************
    sec_3 = df[df['section']=='III.'].copy(deep=True)
    sec_3.reset_index(inplace=True, drop=True)
    subsets = ['1.','2.','3.','4.','5.','6.','7.','8.']
    
    sec_3['section'] = sec_3[1].apply(lambda x: sub_sections(x))
    x='drop'
    x_l=[]
    for i in sec_3['section'].tolist():
        if type(i)==str:
            x=i
        x_l.append(x)
    sec_3['section']=x_l
    sec_3 = sec_3[sec_3['section']!='drop']
    sec_3_1 = sec_3[sec_3['section']=='1.']
    sec_3_1.reset_index(inplace=True, drop=True)
    sec_3_1.columns = sec_3_1.loc[2]
    sec_3_1 = sec_3_1.iloc[4:-1:,:-2:].copy(deep=True)
    col3_1=[u'Raz\xf3n social de la entidad \u2013p\xfablica o privada\u2013 en la que desempe\xf1a o ha desempe\xf1\xf3 la posici\xf3n, cargo o funci\xf3n',
     u'Posici\xf3n, cargo o funci\xf3n que desempe\xf1a o ha desempe\xf1ado en los \xfaltimos cinco a\xf1os',
     u'Pa\xeds en el que est\xe1 constituida la entidad',
     u'Fecha de inicio de la posici\xf3n, cargo o funci\xf3n (DD/MM/AA)',
     u'Fecha de t\xe9rmino de finalizaci\xf3n del cargo o funci\xf3n (DD/MM/AA)',
     u'Remuneraci\xf3n anual neta recibida por el desempe\xf1o en la posici\xf3n, cargo o funci\xf3n.']
    sec_3_1 = sec_3_1[col3_1]
    sec_3_1 = check_empty(sec_3_1)
    sec_3_1.reset_index(inplace=True, drop=True)
    put_nulls(sec_3_1)
    
    sec_3_2 = sec_3[sec_3['section']=='2.']
    sec_3_2.reset_index(inplace=True, drop=True)
    sec_3_2.columns = sec_3_2.loc[2]
    col3_2=['Actividad profesional o empresarial que desempe\xf1a o ha desempe\xf1ado en los \xfaltimos 5 a\xf1os',
            'nan','nan',
    'Nombre del cliente o empresa para la que se prest\xf3 el servicio', 'nan',
    'Pa\xeds en el que se desarroll\xf3 la actividad profesional o empresarial',
    'Fecha de inicio de la presentaci\xf3n del servicio (DD/MM/AA)',
    'Fecha de finalizaci\xf3n de la prestaci\xf3n del servicio (DD/MM/AA)',
    'Contraprestaci\xf3n anual neta recibida por el servicio provisto','keep','2.']
    if(len(sec_3_2.columns)==len(col3_2)):
        sec_3_2.columns = col3_2
    sec_3_2 = sec_3_2.iloc[4:-1:,:-2:]
    sec_3_2.reset_index(inplace=True, drop=True)
    col3_2=[u'Actividad profesional o empresarial que desempe\xf1a o ha desempe\xf1ado en los \xfaltimos 5 a\xf1os',
     u'Nombre del cliente o empresa para la que se prest\xf3 el servicio',
     u'Pa\xeds en el que se desarroll\xf3 la actividad profesional o empresarial',
     u'Fecha de inicio de la presentaci\xf3n del servicio (DD/MM/AA)',
     u'Fecha de finalizaci\xf3n de la prestaci\xf3n del servicio (DD/MM/AA)',
     u'Contraprestaci\xf3n anual neta recibida por el servicio provisto']
    sec_3_2 = sec_3_2[col3_2]
    sec_3_2 = check_empty(sec_3_2)
    sec_3_2.reset_index(inplace=True, drop=True)
    put_nulls(sec_3_2)
    
    sec_3_3 = sec_3[sec_3['section']=='3.']
    sec_3_3.reset_index(inplace=True, drop=True)
    if(sec_3_3.loc[2][0]=='Parentesco (relaci\xf3n entre persona con el declarante)'):
        sec_3_3.columns = sec_3_3.loc[2]
    else:
        sec_3_3.columns = sec_3_3.loc[1]
    #sec_3_3.columns = sec_3_3.loc[2]
    sec_3_3 = sec_3_3.iloc[4:-1:,:-2:]
    sec_3_3.reset_index(inplace=True, drop=True)
    col3_3=[u'Parentesco (relaci\xf3n entre persona con el declarante)',
     u'Actividad, cargo o funci\xf3n desempe\xf1ada',
     u'Sector econ\xf3mico de la entidad donde se desempe\xf1a',
     u'Tipo de actividad de la entidad',
     u'Pa\xeds en el que est\xe1 constituida la entidad o en el que desempe\xf1a la actividad']
    sec_3_3 = sec_3_3[col3_3]
    sec_3_3 = check_empty(sec_3_3)
    sec_3_3.reset_index(inplace=True, drop=True)
    put_nulls(sec_3_3)
    
    # SECCION IV *************************************************************
    sec_4 = df[df['section']=='IV.'].copy(deep=True)
    sec_4.reset_index(inplace=True, drop=True)
    subsets = ['1.','2.','2.1','3.','4.','5.','6.','7.','8.']

    sec_4['section'] = sec_4[1].apply(lambda x: sub_sections(x))
    x='drop'
    x_l=[]
    for i in sec_4['section'].tolist():
        if type(i)==str:
            x=i
        x_l.append(x)
    sec_4['section']=x_l
    sec_4 = sec_4[sec_4['section']!='drop']
    sec_4.reset_index(inplace=True, drop=True)
    
    sec_4_1 = sec_4[sec_4['section']=='1.']
    sec_4_1.reset_index(inplace=True, drop=True)
    sec_4_1.columns = sec_4_1.loc[2]
    sec_4_1 = sec_4_1.iloc[4::,:-2:]
    col4_1=[u'Nombre o raz\xf3n social de la organizaci\xf3n o instuci\xf3n en la que participa',
     u'Posici\xf3n o cargo honorario',
     u'A\xf1o de inicio de su participaci\xf3n en la organizaci\xf3n',
     u'La participaci\xf3n se encuentra',]
    sec_4_1 = sec_4_1[col4_1]
    sec_4_1 = check_empty(sec_4_1)
    sec_4_1.reset_index(inplace=True, drop=True)
    put_nulls(sec_4_1)
    
    sec_4_2 = sec_4[sec_4['section']=='2.']
    sec_4_2.reset_index(inplace=True, drop=True)
    sec_4_2.columns = sec_4_2.loc[2]
    sec_4_2 = sec_4_2.iloc[4::,:-2:]
    col4_2=[u'Nombre o raz\xf3n social de la organizaci\xf3n o instuci\xf3n en la que participa',
     u'Tipo de participaci\xf3n',
     u'A\xf1o de inicio de su participaci\xf3n en la organizaci\xf3n',
     u'La participaci\xf3n se encuentra']
    sec_4_2 = sec_4_2[col4_2]
    sec_4_2 = check_empty(sec_4_2)
    sec_4_2.reset_index(inplace=True, drop=True)
    put_nulls(sec_4_2)
    
    sec_4_2_1 = sec_4[sec_4['section']=='2.1']
    sec_4_2_1.reset_index(inplace=True, drop=True)
    sec_4_2_1.columns = sec_4_2_1.loc[2]
    sec_4_2_1 = sec_4_2_1.iloc[3::,:-4:]
    col4_2_1=[u'Parentesco (relaci\xf3n entre persona con el declarante)',
     u'Nombre o raz\xf3n social de la organizaci\xf3n o instuci\xf3n en la que participa',
     u'Tipo de participaci\xf3n',
     u'A\xf1o de inicio de su participaci\xf3n en la organizaci\xf3n']
    sec_4_2_1 = sec_4_2_1[col4_2_1]
    sec_4_2_1 = check_empty(sec_4_2_1)
    sec_4_2_1.reset_index(inplace=True, drop=True)
    put_nulls(sec_4_2_1)
    
    sec_4_3 = sec_4[sec_4['section']=='3.']
    sec_4_3.reset_index(inplace=True, drop=True)
    sec_4_3.columns = sec_4_3.loc[2]
    sec_4_3 = sec_4_3.iloc[4:-1:,:-2:]
    col4_3=[u'Nombre o raz\xf3n social de la entidad privada que financi\xf3 el viaje',
     u'Personas que realizaron el viaje',
     u'Fecha en que se realiz\xf3 el viaje',
     u'Pa\xeds al que se realiz\xf3 el viaje',
     u'Descripci\xf3n del prop\xf3sito del viaje',
     u'Valor total o estimado total del viaje']
    sec_4_3 = sec_4_3[col4_3]
    sec_4_3 = check_empty(sec_4_3)
    sec_4_3.reset_index(inplace=True, drop=True)
    put_nulls(sec_4_3)
    
    sec_4_4 = sec_4[sec_4['section']=='4.']
    sec_4_4.reset_index(inplace=True, drop=True)
    sec_4_4.columns = sec_4_4.loc[2]
    sec_4_4 = sec_4_4.iloc[4:-1:,:-3:]
    col4_4=[u'Receptor de patrocinio',
     u'Nombre o raz\xf3n social de la entidad que realiz\xf3 el patrocinio o donativo',
     u'Descripci\xf3n del prop\xf3sito o finalidad del patrocinio',
     u'Valor total o estimado del patrocinio o donativo']
    sec_4_4 = sec_4_4[col4_4]
    sec_4_4 = check_empty(sec_4_4)
    sec_4_4.reset_index(inplace=True, drop=True)
    put_nulls(sec_4_4)
    
    sec_4_5 = sec_4[sec_4['section']=='5.']
    sec_4_5.reset_index(inplace=True, drop=True)
    sec_4_5.columns = sec_4_5.loc[2]
    sec_4_5 = sec_4_5.iloc[4:-1:,:-3:]
    col4_5=[u'Declarante o parentesco de la persona que ha emitido el donativo',
     u'Nombre o raz\xf3n social de la entidad que recibi\xf3 el donativo',
     u'Descripci\xf3n del prop\xf3sito o finalidad del patrocinio',
     u'A\xf1o en el que se realiz\xf3 el donativo',
     u'Valor total o estimado del patrocinio o donativo']
    sec_4_5 = sec_4_5[col4_5]
    sec_4_5 = check_empty(sec_4_5)
    sec_4_5.reset_index(inplace=True, drop=True)
    put_nulls(sec_4_5)

    # SECCION V ********************************************************************
    sec_5 = df[df['section']=='V.']
    sec_5.reset_index(inplace=True, drop=True)
    sec_5.columns = sec_5.loc[2]
    sec_5 = sec_5.loc[3::,::].copy(deep=True)
    sec_5.reset_index(inplace=True, drop=True)
    sec_5 = sec_5.iloc[::,:1:]
    sec_5_final = sec_5
    sec_5_final = check_empty(sec_5_final)
    put_nulls(sec_5_final)

    # SECCION VI *******************************************************************
    sec_6 = df[df['section']=='VI.']
    sec_6.reset_index(inplace=True, drop=True)
    sec_6 = sec_6.iloc[2:-2:,:-5:]
    sec_6.columns = sec_6.loc[2]
    sec_6.drop(2, axis=0, inplace=True)
    col_6 = [u'No. de nota',
     u'Secci\xf3n a la que hace referencia',
     u'Subsecci\xf3n a la que hace referencia',
     u'Descripci\xf3n de la nota aclaratoria']
    sec_6 = sec_6[col_6].copy(deep=True)
    sec_6.reset_index(inplace=True, drop=True)
    sec_6['tag'] = sec_6.apply(lambda row: all_6_nan(row), axis=1)
    sec_6 = sec_6[sec_6['tag'] =='keep'].copy(deep=True)
    sec_6.drop('tag', axis=1,inplace=True)
    sec_6_final = sec_6
    sec_6_final = check_empty(sec_6_final)
    sec_6_final.reset_index(inplace=True, drop=True)
    put_nulls(sec_6_final)

    # SECCION CREANDO JSON ***********************************************************
    table_list = {'sec_1':sec_1_final, 
    'sec_2_1':sec_2_1, 'sec_2_1': sec_2_1_1,'sec_2_2':sec_2_2, 'sec_2_2_1':sec_2_2_1,'sec_2_3':sec_2_3,'sec_2_3_1':sec_2_3_1,'sec_2_4':sec_2_4,'sec_2_5':sec_2_5, 
    'sec_3_1':sec_3_1, 'sec_3_2':sec_3_2, 'sec_3_3':sec_3_3, 
    'sec_4_1':sec_4_1, 'sec_4_2':sec_4_2, 'sec_4_2_1':sec_4_2_1, 'sec_4_3':sec_4_3, 'sec_4_4':sec_4_4, 'sec_4_5':sec_4_5,
    'sec_5':sec_5_final,
    'sec_6':sec_6_final}

    table_list = {'sec_1':sec_1_final, 
    'sec_2_1':sec_2_1, 'sec_2_1': sec_2_1_1,'sec_2_2':sec_2_2, 'sec_2_2_1':sec_2_2_1,'sec_2_3':sec_2_3,'sec_2_3_1':sec_2_3_1,'sec_2_4':sec_2_4,'sec_2_5':sec_2_5, 
    'sec_3_1':sec_3_1, 'sec_3_2':sec_3_2, 'sec_3_3':sec_3_3, 
    'sec_4_1':sec_4_1, 'sec_4_2':sec_4_2, 'sec_4_2_1':sec_4_2_1, 'sec_4_3':sec_4_3, 'sec_4_4':sec_4_4, 'sec_4_5':sec_4_5,
    'sec_5':sec_5_final,
    'sec_6':sec_6_final}

    the_json = {'persona': {sec_1_final['Nombre:'].values[0] + '_' + str(df[9][0]).split(' ')[0]:
         {'sec_1':sec_1_final.to_json(orient='records'),
          'sec_2':{
              'sec_2_1':sec_2_1.to_json(orient='records'), 'sec_2_1_1': sec_2_1_1.to_json(orient='records'),'sec_2_2':sec_2_2.to_json(orient='records'), 
              'sec_2_2_1':sec_2_2_1.to_json(orient='records'),'sec_2_3':sec_2_3.to_json(orient='records'),'sec_2_3_1':sec_2_3_1.to_json(orient='records'),
              'sec_2_4':sec_2_4.to_json(orient='records'),'sec_2_5':sec_2_5.to_json(orient='records'), 
          },
          'sec_3':{
            'sec_3_1':sec_3_1.to_json(orient='records'), 'sec_3_2':sec_3_2.to_json(orient='records'), 'sec_3_3':sec_3_3.to_json(orient='records'), 
          },
          'sec_4':{
              'sec_4_1':sec_4_1.to_json(orient='records'), 'sec_4_2':sec_4_2.to_json(orient='records'), 'sec_4_2_1':sec_4_2_1.to_json(orient='records'), 
              'sec_4_3':sec_4_3.to_json(orient='records'), 'sec_4_4':sec_4_4.to_json(orient='records'),  'sec_4_5':sec_4_5.to_json(orient='records'),
          },
        'sec_5':sec_5_final.to_json(orient='records'),
         'sec_6':sec_6_final.to_json(orient='records'),
         }
        }
    }
    
    with open('Output/Ingresos/jsons/ingresos_'+sec_1_final['Nombre:'].values[0].replace(' ','')+'.json', 'w') as fp:
        json.dump(the_json, fp)

In [8]:
def find_DI_xlsx(cadena):
    patron = re.compile('^(.*?)(_DI| DI|DI |declaracion_intereses|DI_)(.*?)(\.xlsx)$')
    if(patron.search(cadena)):
        print(cadena)
        crea_json(cadena)

In [9]:
directorio_raiz =  pathlib.Path('Data/3de3_CDMX/')

for archivos_1 in directorio_raiz.iterdir():
    directorio_2=str(archivos_1)
    directorio_2=pathlib.Path(directorio_2)
    for archivos_2 in directorio_2.iterdir():
        directorio_3=str(archivos_2)
        directorio_3_aux=str(directorio_3)
        if(directorio_3_aux.count('pdf')==0 and directorio_3_aux.count('xlsx')==0):
            directorio_3 = pathlib.Path(directorio_3)
            for archivos_3 in directorio_3.iterdir():
                excel=str(archivos_3)
                if(excel.count('xls')==1):
                    find_DI_xlsx(str(archivos_3))
        else:
            find_DI_xlsx(str(directorio_3_aux))

Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/03_Beatriz_Adriana_Olivares_Pinal/02 DI BAOP.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/03_Esthela_Damian_Peralta/02_DI_EDP.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/03_Rafael_Perez_Laurrabaquio/02_DI_RPL.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/02_Jaime_Ruben_Morales_Beltran/01_DI_JRMB.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/02_Araceli_Damian_Gonzalez/02_DI_ADG.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/01_Almudena_Ocejo_Rojo/02_DI_AOR.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/03_Rafael_Vargas_Morales/02_DI_RMV.xlsx
Data/3de3_CDMX/Procuraduria_de_Justicia/02_Rodrigo_de_la_Riva/02_DI_RRR.xlsx
Data/3de3_CDMX/Coordinacion_General_de_Comunicacion_Social/01_Cesar_Ivan_Escalante_Ruiz/02_DI_CIER.xlsx
Data/3de3_CDMX/Consejo_Ciudadano/declaracion_intereses.xlsx
Data/3de3_CDMX/Secretaria_para_la_Autonomia_de_la_M

In [10]:
from pprint import pprint

json_completo=[]
directorio_jsons =  pathlib.Path('Output/Ingresos/jsons/')

for archivo in directorio_jsons.iterdir():
    with open(str(archivo)) as f:
        data = json.load(f)
        json_completo.append(data)
#pprint(json_completo)

with open('Output/Ingresos/json_completo/json_completo.json', 'w') as fp:
    json.dump(json_completo, fp)